In [178]:
## This sets some printing options
## that make output look nicer.

from pprint import pprint as print
import pandas as pd
import requests
from lxml import html,etree

pd.set_option('display.width', 133)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', 5)

In [179]:
dnd_domain = 'http://dnd5e.wikidot.com'
choice_domain= 'spells'

dnd_url = (dnd_domain
                + "/"
                + choice_domain)

print(dnd_url)

spells = requests.get(dnd_url)

spells.headers['Content-Type']

'http://dnd5e.wikidot.com/spells'


'text/html; charset=utf-8'

In [180]:
spells_html = html.fromstring(spells.text)

#html.open_in_browser(spells_html, encoding = 'UTF-8')


In [181]:
spells_list_html = spells_html.xpath('//*[@id="wiki-tabview-8d4b350373d046fa0c6b0d6db5cd27d6"]/div')
#spells_list_html = spells_html.xpath('//*[@id="wiki-tab-0-0"]/div/table/tbody/tr[2]/td[1]/a')

print(spells_list_html)

[<Element div at 0x225992151d0>]


In [182]:
second_spell_html = spells_list_html[0]
#html.open_in_browser(second_spell_html, encoding = 'UTF-8')

In [229]:
spell_list = []
for i in range(9):
    element = second_spell_html.xpath(f'//*[@id="wiki-tab-0-{i}"]/div/table/tr/td/a/@href')
    for el in element:
        spell_list.append(el)
print(len(spell_list))
#print(element)
spell_url


558


In [7]:
import re
from collections import OrderedDict
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import requests


SpellListUrl = requests.get('http://dnd5e.wikidot.com/spells')
source1 = BeautifulSoup(SpellListUrl.content, "html.parser")

# the lambda function checks if the href attribute contains "spell" and does not contain "spells".
# If both conditions are true, it returns True, indicating that the tag should be included in the result.
# courtesy of Chatgpt

# allSpellNames = []
# allSpellLinks = []
# for a in source1.find_all(href=lambda href: href and "spell" in href and "spells" not in href):
#     allSpellLinks.append(a['href'])
#     allSpellNames.append(a.text)




def pull_spellinfo(SpellContent):
    temp = []

    table = SpellContent.find('div',attrs={'id': 'page-content'}).find('table') #use a 'try' here and if if positive then go into seperate function for finding table data
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)

    df = pd.DataFrame(res, columns=["A", "B", "C", "D",'E','F'])
    print(df.style())


    temp.append(SpellContent.find('div',attrs={'class':'page-title page-header'}).text)
    for l in SpellContent.find('div',attrs={'id': 'page-content'}).strings:
        temp.append(l)
    Spellinfolist = list(filter(lambda a: a != '\n', temp))
    return Spellinfolist
    # Need to do post-processing with a table here ( make separate function for this)
    # also maybe for list as well

def clean_spellinfo(testwords,spell_details):
    #testwords = testwords[1].split()
    for x in range(len(testwords)):
        testwords[x] = re.sub(r'Source: ', '', testwords[x], flags=re.IGNORECASE)
    spell_details['Spell'] = testwords[0]
    keyword = testwords.index("Casting Time:")
    #print(testwords[keyword+1])





testList=['/spell:acid-splash','/spell:alarm','/spell:antagonize','/spell:druidcraft','/spell:summon-celestial']

spell_details = OrderedDict ({'Spell': '','Source':'', 'School': '','Level':'', 'Casting Time': '','Range':'','Components':'','Duration':'', 'Description': '', 'At Higher Levels': '','Spell lists': '','Link':'' })

#for i in range(len(testList)):
sourceURL = ('http://dnd5e.wikidot.com' + testList[4])
SpellUrlGet = requests.get(sourceURL)
SpellContent = BeautifulSoup(SpellUrlGet.content, "html.parser")
Spellinfolist = pull_spellinfo(SpellContent)
clean_spellinfo(Spellinfolist,spell_details)
#print(spell_details)

TypeError: 'Styler' object is not callable